In [ ]:
import st2
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np
import datetime


In [ ]:
from st2 import apply_pivots, custom_resample

In [ ]:
min5_data =  st2.load_data(r'/Users/suganeshr/Trading/Trading_codes/nifty_200/ABB.csv')

In [ ]:
indexed_5min = min5_data.set_index('Datetime')
indexed_5min.index = pd.to_datetime(indexed_5min.index)

In [ ]:
hourly_data = custom_resample(indexed_5min, freq='h').dropna()

In [ ]:
hourly_data.index

In [ ]:
daily_data = custom_resample(hourly_data,freq='D').dropna()


In [ ]:
daily_data = apply_pivots(daily_data)

In [ ]:
hourly_data['Date']  = hourly_data.index.date

In [ ]:
hourly_data['datetime'] =  hourly_data.index

In [ ]:
merged_Data = pd.merge(hourly_data, daily_data[['Date', 'PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', '%CPR']], on='Date')



In [ ]:
merged_Data.set_index('datetime',inplace=True)

In [ ]:
merged_Data.dropna(inplace= True)

In [ ]:
merged_Data

In [ ]:
def calculate_indicators(df):
    df['rsi'] = ta.rsi(df['Close'], length=14).astype('float64')
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
    print(type(df['mfi'].iloc[1]))

    macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
    df['macd'] = macd['MACD_12_26_9'].astype('float64')
    df['macd_signal'] = macd['MACDs_12_26_9'].astype('float64')

    bb = ta.bbands(df['Close'], length=20, std=2)
    df['bb_upper'] = bb['BBU_20_2.0'].astype('float64')
    df['bb_lower'] = bb['BBL_20_2.0'].astype('float64')

    # Calculate %b (Percent B) for Bollinger Bands
    df['percent_b'] = ((df['Close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower']) * 100).astype('float64')

    stoch = ta.stoch(df['High'], df['Low'], df['Close'], k=14, d=3)
    df['stoch_k'] = stoch['STOCHk_14_3_3'].astype('float64')
    df['stoch_d'] = stoch['STOCHd_14_3_3'].astype('float64')

    adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
    df['adx'] = adx['ADX_14'].astype('float64')
    return  df


In [ ]:
data = calculate_indicators(merged_Data)

In [ ]:
"""
        condition1 = self.rsi[0] > 70
        condition2 = self.mfi[0] > 80
        condition3 = self.macd[0] > self.macd_signal[0]
        condition4 = self.percent_b[0] > 80
        condition5 = self.stoch_k[0] > 80 and self.stoch_d[0] > 80
        condition6 = self.adx[0] > 20
        condition7 = self.data.close[0] > self.pp[0]
        condition8 = self.data.close[0] > self.r2[0]

"""
start_time =datetime.time(10, 14)
end_time = datetime.time(14, 16)
close_time = datetime.time(15,10)

buy_condition = (data.rsi > 70) & (data.mfi > 80) & (data.macd > data.macd_signal) & (data.percent_b > 80) & (data.stoch_k > 80) & (data.stoch_d > 80) &(data.adx > 20) & (data.Close > data.R2) &(data.index.time > start_time) & (data.index.time < end_time) 
sell_condition = (data.Close > data.R3) | (data.Close < data.TC) |(data.index.time >= close_time)

In [ ]:
conditions = [buy_condition,sell_condition]

In [ ]:
choices = ['Buy','Sell']

In [ ]:



data['Signal'] = np.select(conditions, choices)

In [ ]:
data['Shifted_close'] = data['Close'].shift()

In [ ]:
import math


def Backtest(df):
    position = False
    buydates = []
    buyprices = []
    selldates = []
    sellprices = []
    qtys = []
    capital = 100000
    for index, row in df.iterrows():
        if row['Signal'] == 'Buy' and not position:
            position = True
            buydates.append(index)
            buyprices.append(row['Close'])
            qtys.append(math.floor(capital / row['Close']))
            
            
        
        if position:
            if row['Signal'] == 'Sell' or row['Shifted_close'] < 0.99 * buyprices[-1]  or row['Shifted_close'] > 1.03 * buyprices[-1] :
                selldates.append(index)
                sellprices.append(row['Close'])
                position = False

    # Ensure lists are of equal length by filling with NaN
    max_len = max(len(buydates), len(selldates))
    buydates.extend([np.nan] * (max_len - len(buydates)))
    buyprices.extend([np.nan] * (max_len - len(buyprices)))
    selldates.extend([np.nan] * (max_len - len(selldates)))
    sellprices.extend([np.nan] * (max_len - len(sellprices)))
    qtys.extend([np.nan] * (max_len - len(qtys)))
    
    return buydates, buyprices, selldates, sellprices, qtys


In [ ]:
Backtest(data)

In [ ]:
# Assuming you have already defined and run the Backtest function
buydates, buyprices, selldates, sellprices,qtys = Backtest(data)

# Create a trade history dictionary
trade_history = {
    'Buy Date': buydates,
    'Buy Price': buyprices,
    'Sell Date': selldates,
    'Sell Price': sellprices,
    'Quantity' :qtys
}

# Convert the dictionary to a DataFrame
trade_history_df = pd.DataFrame(trade_history)


# Display the DataFrame
print(trade_history_df)

In [ ]:
trade_history_df['Return'] = (trade_history_df['Sell Price'] - trade_history_df['Buy Price']) / trade_history_df['Buy Price']

In [ ]:
trade_history_df['Days'] = (trade_history_df['Sell Date'] - trade_history_df['Buy Date'])
trade_history_df['PnL'] = trade_history_df['Sell Price'] - trade_history_df['Buy Price']
trade_history_df['Realized Profit'] = trade_history_df['PnL'] * trade_history_df['Quantity']
trade_history_df['Cum Profit'] = trade_history_df['Realized Profit'].cumsum()
trade_history_df

In [ ]:
len(trade_history_df[trade_history_df['PnL'] > 0])

In [ ]:
trade_history_df.plot(x=trade_history_df.index, y='Cum Profit', kind='line')